In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
import getpass
import csv

ModuleNotFoundError: No module named 'psycopg2'

## 1. Extract mortality

#### Alternative 1: sqlite3

In [ ]:
import sqlite3

##############################################################################
####################### CONNECT TO MIMIC #####################################
connection = sqlite3.connect("C:\\Users\\Maria\\Desktop\\Projects Data Scripts\\MIMIC\\data\\mimic3.db")

#We can verify we successfully created our connection object by running:
print(connection.total_changes)
# Be sure to close the connection
#con.close()

# Create our test query
test_query = """
SELECT subject_id, hadm_id, admittime, dischtime, admission_type, diagnosis
FROM admissions
"""

# Run the query and assign the results to a variable
test = pd.read_sql_query(test_query,connection)

print("TEST:", test.head())
##############################################################################
##################### QUERYING, MORTALITY COHORT #############################

query = """

WITH icu_patients AS
(
SELECT icu.subject_id, icu.hadm_id, icu.icustay_id, pat.DOB, pat.gender, icu.intime 
, (JulianDay(OUTTIME) - JulianDay(INTIME)) as icu_length_of_stay
, (JulianDay(icu.INTIME) - JulianDay(pat.DOB))/ 364.242 as age
, adm.hospital_expire_flag , icu.outtime 
, RANK() OVER (PARTITION BY icu.subject_id ORDER BY icu.intime) AS icustay_id_order 


FROM icustays icu
INNER JOIN patients pat
  ON icu.subject_id = pat.subject_id
INNER JOIN admissions adm
    ON adm.subject_id = icu.subject_id
    AND adm.hadm_id = icu.hadm_id
)


SELECT
    fa.subject_id, fa.hadm_id, fa.icustay_id, fa.icustay_id_order, fa.intime as icu_intime, fa.outtime as icu_outtime, fa.DOB, fa.GENDER, fa.age as patient_age, fa.hospital_expire_flag as mortality, fa.icu_length_of_stay
  , CASE 
        WHEN fa.icu_length_of_stay < 2 then 1
    ELSE 0 END
        as exclusion_los
  , CASE
        WHEN fa.age < 18 then 1
    ELSE 0 END
        as exclusion_age
FROM icu_patients fa

"""

icu = pd.read_sql_query(query, connection)

In [ ]:
mortality_cohort = icu.copy()

#### Alternative 2: PostgreSQL

In [2]:
# Set up postgres connection
conn = psycopg2.connect(
    database='mimic', 
    user='zhwa9764', 
    password=getpass.getpass("Enter postgres password"), 
    host='127.0.0.1', 
    port='5432',
    options=f'-c search_path=mimiciii')

Enter postgres password········


In [3]:
##################### QUERYING, MORTALITY COHORT #############################
query = """
WITH icu_patients AS
(
SELECT icu.subject_id, icu.hadm_id, icu.icustay_id, pat.DOB, pat.gender, icu.intime 
, DATE_PART('day', OUTTIME - INTIME) as icu_length_of_stay
, DATE_PART('day', icu.INTIME - pat.DOB) / 364.242 as age
, adm.hospital_expire_flag , icu.outtime 
, RANK() OVER (PARTITION BY icu.subject_id ORDER BY icu.intime) AS icustay_id_order 
FROM icustays icu
INNER JOIN patients pat
  ON icu.subject_id = pat.subject_id
INNER JOIN admissions adm
    ON adm.subject_id = icu.subject_id
    AND adm.hadm_id = icu.hadm_id
)
SELECT
    fa.subject_id, fa.hadm_id, fa.icustay_id, fa.icustay_id_order, fa.intime as icu_intime, fa.outtime as icu_outtime, fa.DOB, fa.GENDER, fa.age as patient_age, fa.hospital_expire_flag as mortality, fa.icu_length_of_stay
  , CASE 
        WHEN fa.icu_length_of_stay < 2 then 1
    ELSE 0 END
        as exclusion_los
  , CASE
        WHEN fa.age < 18 then 1
    ELSE 0 END
        as exclusion_age
FROM icu_patients fa
"""

mortality_cohort = pd.read_sql(query, conn)

### Data preprocessing

In [4]:
mortality_cohort.shape

(61532, 13)

In [5]:
mortality_cohort.head()

,subject_id,hadm_id,icustay_id,icustay_id_order,icu_intime,icu_outtime,dob,gender,patient_age,mortality,icu_length_of_stay,exclusion_los,exclusion_age
0,2,163353,243653,1,2138-07-17 21:20:07,2138-07-17 23:32:21,2138-07-17,M,0.000000,0,0.0,1,1
1,3,145834,211552,1,2101-10-20 19:10:11,2101-10-26 20:43:09,2025-04-11,M,76.734698,0,6.0,0,0
2,4,185777,294638,1,2191-03-16 00:29:31,2191-03-17 16:46:31,2143-05-12,F,47.976345,0,1.0,1,0
3,5,178980,214757,1,2103-02-02 06:04:24,2103-02-02 08:06:00,2103-02-02,M,0.000000,0,0.0,1,1
4,6,107064,228232,1,2175-05-30 21:30:54,2175-06-03 13:39:54,2109-06-21,F,66.120876,0,3.0,0,0


In [6]:
pd.value_counts(mortality_cohort['hadm_id']) # same addmission ids occured more than once: (one admission id could be link to mutiple icu stays)

163281    7
186706    7
175448    6
123178    6
109520    6
         ..
127444    1
117203    1
119248    1
176572    1
131072    1
Name: hadm_id, Length: 57786, dtype: int64

### Extract only the adults, and stay length larger than 1 day

In [7]:
adults = mortality_cohort[mortality_cohort['exclusion_age'] == 0]

adults = adults.loc[adults['icu_length_of_stay'] >= 1]

In [8]:
adults.shape

(45256, 13)

In [9]:
adults.head()

,subject_id,hadm_id,icustay_id,icustay_id_order,icu_intime,icu_outtime,dob,gender,patient_age,mortality,icu_length_of_stay,exclusion_los,exclusion_age
1,3,145834,211552,1,2101-10-20 19:10:11,2101-10-26 20:43:09,2025-04-11,M,76.734698,0,6.0,0,0
2,4,185777,294638,1,2191-03-16 00:29:31,2191-03-17 16:46:31,2143-05-12,F,47.976345,0,1.0,1,0
4,6,107064,228232,1,2175-05-30 21:30:54,2175-06-03 13:39:54,2109-06-21,F,66.120876,0,3.0,0,0
8,9,150750,220597,1,2149-11-09 13:07:02,2149-11-14 20:52:14,2108-01-26,M,41.903460,1,5.0,0,0
10,11,194540,229441,1,2178-04-16 06:19:32,2178-04-17 20:21:05,2128-02-22,F,50.285250,0,1.0,1,0


In [10]:
pd.value_counts(adults['mortality'])

0    39895
1     5361
Name: mortality, dtype: int64

In [11]:
# check if there is any duplicate in the table
sum(adults.duplicated())

0

### Filter to get positive and negative mortality patients separately

In [12]:
# select the max of icustay order (last admission stay) with the positive mortality 
# pos_filter = adults[adults['mortality']==1].groupby(['subject_id'])['icustay_id_order'].idxmax() 
patient_mortality = (adults.groupby(['subject_id'])['mortality'].max() == 1).reset_index()
pos_patient_set = set(patient_mortality[patient_mortality['mortality']==True]['subject_id'])

pos_filter = adults[adults['subject_id'].isin(pos_patient_set)].groupby(['subject_id'])['icustay_id_order'].idxmax() 
pos_adults = adults.loc[pos_filter]

pos_adults.head()

,subject_id,hadm_id,icustay_id,icustay_id_order,icu_intime,icu_outtime,dob,gender,patient_age,mortality,icu_length_of_stay,exclusion_los,exclusion_age
8,9,150750,220597,1,2149-11-09 13:07:02,2149-11-14 20:52:14,2108-01-26,M,41.903460,1,5.0,0,0
11,12,112213,232669,1,2104-08-08 02:08:17,2104-08-15 17:22:25,2032-03-24,M,72.572630,1,7.0,0,0
20,21,111970,216859,2,2135-01-30 20:53:34,2135-02-08 05:38:46,2047-04-04,M,88.065078,1,8.0,0,0
30,31,128652,254478,1,2108-08-22 23:28:42,2108-08-30 21:59:20,2036-05-17,M,72.462813,1,7.0,0,0
56,56,181711,275642,1,2104-01-02 02:02:39,2104-01-03 22:25:29,1804-01-02,F,300.824726,1,1.0,1,0


In [13]:
pos_adults.shape

(4774, 13)

In [14]:
pos_adm_set = set(pos_adults['hadm_id'])

In [15]:
# select the mix of icustay order (first admission stay) with the negative mortality 
# neg_filter = adults[adults['mortality']==0].groupby(['subject_id'])['icustay_id_order'].idxmin() 

neg_patient_set = set(patient_mortality[patient_mortality['mortality']==False]['subject_id'])

neg_filter = adults[adults['subject_id'].isin(neg_patient_set)].groupby(['subject_id'])['icustay_id_order'].idxmin() 
neg_adults = adults.loc[neg_filter]

neg_adults.head()

,subject_id,hadm_id,icustay_id,icustay_id_order,icu_intime,icu_outtime,dob,gender,patient_age,mortality,icu_length_of_stay,exclusion_los,exclusion_age
1,3,145834,211552,1,2101-10-20 19:10:11,2101-10-26 20:43:09,2025-04-11,M,76.734698,0,6.0,0,0
2,4,185777,294638,1,2191-03-16 00:29:31,2191-03-17 16:46:31,2143-05-12,F,47.976345,0,1.0,1,0
4,6,107064,228232,1,2175-05-30 21:30:54,2175-06-03 13:39:54,2109-06-21,F,66.120876,0,3.0,0,0
10,11,194540,229441,1,2178-04-16 06:19:32,2178-04-17 20:21:05,2128-02-22,F,50.285250,0,1.0,1,0
12,13,143045,263738,1,2167-01-08 18:44:25,2167-01-12 10:43:31,2127-02-27,F,39.973424,0,3.0,0,0


In [16]:
neg_adults.shape

(28790, 13)

In [17]:
pd.value_counts(neg_adults['hadm_id'])

141307    1
190038    1
167323    1
173470    1
190189    1
         ..
127659    1
182956    1
180909    1
187054    1
131072    1
Name: hadm_id, Length: 28790, dtype: int64

In [18]:
neg_adm_set = set(neg_adults['hadm_id'])

#### Check if there is any patient appearing in both negative or positive sets

In [19]:
set1 = set(pos_adults['subject_id'].reset_index(drop=True))
set2 = set(neg_adults['subject_id'].reset_index(drop=True))

In [20]:
len(list(set1 & set2))

0

## 2. Get clinical texts

In [21]:
texts = pd.read_sql(
    """
    SELECT hadm_id, subject_id, chartdate, category, description, text
    FROM noteevents;
    """, conn)

In [22]:
texts.head()

,hadm_id,subject_id,chartdate,category,description,text
0,167853.0,22532,2151-08-04,Discharge summary,Report,Admission Date: [**2151-7-16**] Dischar...
1,107527.0,13702,2118-06-14,Discharge summary,Report,Admission Date: [**2118-6-2**] Discharg...
2,167118.0,13702,2119-05-25,Discharge summary,Report,Admission Date: [**2119-5-4**] D...
3,196489.0,13702,2124-08-18,Discharge summary,Report,Admission Date: [**2124-7-21**] ...
4,135453.0,26880,2162-03-25,Discharge summary,Report,Admission Date: [**2162-3-3**] D...


In [23]:
texts.shape

(2083180, 6)

In [24]:
pd.value_counts(texts['category'])

Nursing/other        822497
Radiology            522279
Nursing              223556
ECG                  209051
Physician            141624
Discharge summary     59652
Echo                  45794
Respiratory           31739
Nutrition              9418
General                8301
Rehab Services         5431
Social Work            2670
Case Management         967
Pharmacy                103
Consult                  98
Name: category, dtype: int64

In [25]:
category_set = {'Nursing/other', 'Radiology', 'Nursing', 'ECG', 'Physician '}

filtered_texts = texts[texts['category'].isin(category_set)]


In [26]:
filtered_texts.head()

,hadm_id,subject_id,chartdate,category,description,text
52134,119078.0,13289,2103-07-28,ECG,Report,Sinus tachycardia\nShort PR interval\nPossible...
93978,187620.0,6541,2133-03-13,ECG,Report,Sinus bradycardia\n\nNormal ECG except for rat...
93984,156154.0,26175,2112-05-03,ECG,Report,Compared to the previous tracing of [**2112-4-...
94000,NaN,5445,2128-08-27,ECG,Report,"Sinus bradycardia, rate 57. Left atrial abnorm..."
94001,170426.0,21892,2138-01-28,ECG,Report,"Sinus tachycardia. Otherwise, normal tracing. ..."


In [27]:
pd.value_counts(filtered_texts['category'])

Nursing/other    822497
Radiology        522279
Nursing          223556
ECG              209051
Physician        141624
Name: category, dtype: int64

In [28]:
texts_by_adm = filtered_texts.groupby(['hadm_id']).apply(lambda x: x['text']).reset_index(level=[1], drop=True).reset_index()

In [29]:
joined_texts = texts_by_adm.groupby(['hadm_id'])['text'].apply(lambda x: ','.join(x)).reset_index()

In [30]:
joined_texts.head()

,hadm_id,text
0,100001.0,[**2117-9-11**] 11:12 AM\n CHEST (PA & LAT) ...
1,100003.0,Sinus rhythm\nProlonged QT interval is nonspec...
2,100006.0,Sinus tachycardia\nLeft axis deviation - anter...
3,100007.0,Sinus rhythm\nAtrial premature complex\nConsid...
4,100009.0,Sinus bradycardia. Left atrial abnormality. ...


In [31]:
joined_texts.shape

(58006, 2)

In [32]:
pos_texts = joined_texts[joined_texts['hadm_id'].isin(pos_adm_set)]
pos_texts['label'] = [1 for i in range(pos_texts.shape[0])]

neg_texts = joined_texts[joined_texts['hadm_id'].isin(neg_adm_set)]
neg_texts['label'] = [0 for i in range(neg_texts.shape[0])]


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [33]:
pos_texts.shape

(4602, 3)

In [34]:
neg_texts.shape

(28639, 3)

In [35]:
concat_texts = pd.concat([pos_texts, neg_texts])
concat_texts = concat_texts.sample(frac=1, random_state=39).reset_index(drop=True)
concat_texts

,hadm_id,text,label
0,178137.0,Sinus bradycardia. A-V conduction delay. Q-T i...,0
1,118804.0,Baseline artifact\nSinus rhythm\nRight bundle ...,1
2,182422.0,Atrial fibrillation with possible flutter with...,0
3,152109.0,Mobitz II second degree A-V block with non-con...,0
4,122660.0,Sinus rhythm. Normal tracing. No previous tr...,0
...,...,...,...
33236,112156.0,Atrial fibrillation. Left anterior fascicular...,0
33237,171365.0,Sinus rhythm. Normal ECG. No previous tracing ...,0
33238,193197.0,Sinus rhythm\nEarly precordial QRS transition ...,0
33239,109012.0,Sinus rhythm.\nLeft axis deviation\nIV conduct...,0
